In [1]:
import cfbd
from cfbd.rest import ApiException
from pprint import pprint
import pandas as pd
import sqlite3
import DataFunctions
import numpy as np


configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = '3WCU5V2X05Rvh60ZxUG8FarJN4s2D1lcd2c2r6Kz/qL1Y3tVBJtWsuNATnzHRV2h'
configuration.api_key_prefix['Authorization'] = 'Bearer'

## Data Preprocessing

Games DF

In [2]:
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
year=2022
division="fbs"

# no longer need, right?
# try:
#     api_response = api_instance.get_games(year=year,division=division)
#     print(len(api_response))
# except:
#     print(ApiException)
    

In [3]:
gamelist = DataFunctions.get_fbs_games(api_instance=api_instance,year=2022)
# len(gamelist)

In [4]:
games_df = DataFunctions.df_from_games(gamelist=gamelist)
# games_df.head()

In [5]:
# conn = sqlite3.connect("CollegeFootball.db")
# games_df.to_sql("games",conn,if_exists="replace",index=False)
# conn.close()


In [6]:
# we can see with this code that we can extract DF from SQL db

# conn = sqlite3.connect("CollegeFootball.db")
# trialdf = pd.read_sql_query("SELECT * FROM games",conn)
# conn.close()
# print(trialdf.shape)
# trialdf.head()


Betting lines DF

In [7]:
api_instance1 = cfbd.BettingApi(cfbd.ApiClient(configuration))

In [8]:
conferences=[]
for game in gamelist:
    conferences.append(game.away_conference)
conferences=set(conferences)

conferences

{'ACC',
 'American Athletic',
 'Big 12',
 'Big Ten',
 'Conference USA',
 'FBS Independents',
 'Mid-American',
 'Mountain West',
 'Pac-12',
 'SEC',
 'Sun Belt'}

In [9]:
betting_list=DataFunctions.get_fbs_betting(api_instance=api_instance1,year=2022,conferences=conferences)

In [10]:
betting_df=DataFunctions.df_betting_lines(betting_list)
betting_df

#warning seems to be since betting line -0.0 for one of the games but still stores value as 0 so should be fine

,id,av_spread,av_total
0,401411101,-24.375,51.375
1,401404079,7.125,65.625
2,401403893,1.500,51.000
3,401404080,2.500,45.000
4,401404081,13.625,53.000
...,...,...,...
733,401403935,12.500,50.500
734,401404029,21.125,61.125
735,401416641,-2.750,55.750
736,401404036,11.500,47.000


In [11]:
# conn=sqlite3.connect("CollegeFootball.db")
# betting_df.to_sql("betting_lines",conn,if_exists="replace",index=False)
# conn.close()

In [12]:
# #sample merge dataframe

# cmd=\
# """
# SELECT G.away_team,G.away_points,G.home_points,G.home_team,B.av_spread,B.av_total
# FROM games G
# INNER JOIN betting_lines B ON G.id=B.id
# """
# conn=sqlite3.connect("CollegeFootball.db")
# test_merge=pd.read_sql_query(cmd,conn)
# conn.close() # do not forget this after query
# test_merge

Team stats per season DF

In [13]:
api_instance2 = cfbd.StatsApi(cfbd.ApiClient(configuration))

In [14]:
teamstats = api_instance2.get_advanced_team_season_stats(year=year)

In [15]:
stats_df = DataFunctions.df_team_advstats(teamstats=teamstats)

In [16]:
cols = ['team','season','conference','Offensive ppa','Offensive success_rate',
        'Offensive explosiveness','Offensive power_success',
        'Offensive stuff_rate','Offensive line_yards',
        'Defensive ppa','Defensive success_rate',
        'Defensive explosiveness','Defensive power_success',
        'Defensive stuff_rate','Defensive line_yards',
        'Offensive havoc total','Offensive rushing_plays ppa',
        'Offensive rushing_plays success_rate',
        'Offensive rushing_plays explosiveness',
        'Offensive passing_plays ppa',
        'Offensive passing_plays success_rate',
        'Offensive passing_plays explosiveness',
        'Defensive havoc total','Defensive rushing_plays ppa',
        'Defensive rushing_plays success_rate',
        'Defensive rushing_plays explosiveness',
        'Defensive passing_plays ppa',
        'Defensive passing_plays success_rate',
        'Defensive passing_plays explosiveness']
stats_df1 = DataFunctions.df_stats_needed(stats_df,cols)

In [17]:
stats_df1.shape

(131, 29)

In [18]:
stats_df1.head()

,team,season,conference,Offensive ppa,Offensive success_rate,Offensive explosiveness,Offensive power_success,Offensive stuff_rate,Offensive line_yards,Defensive ppa,...,Offensive passing_plays ppa,Offensive passing_plays success_rate,Offensive passing_plays explosiveness,Defensive havoc total,Defensive rushing_plays ppa,Defensive rushing_plays success_rate,Defensive rushing_plays explosiveness,Defensive passing_plays ppa,Defensive passing_plays success_rate,Defensive passing_plays explosiveness
0,Air Force,2022,Mountain West,0.297878,0.480570,1.132888,0.825000,0.093979,3.557269,0.123016,...,0.505886,0.382022,2.614222,0.196944,0.097045,0.394737,0.969436,0.255682,0.425455,1.410758
1,Akron,2022,Mid-American,0.113974,0.426016,1.185226,0.735294,0.154185,3.144493,0.269248,...,0.231801,0.418848,1.459991,0.199301,0.156959,0.446541,1.005307,0.449689,0.480000,1.656349
2,Alabama,2022,SEC,0.305324,0.471508,1.375730,0.722222,0.194954,2.970183,0.028379,...,0.394611,0.479303,1.637813,0.180089,0.078791,0.407583,0.952710,0.012309,0.348936,1.379126
3,Appalachian State,2022,Sun Belt,0.259159,0.456019,1.295246,0.705882,0.127660,3.395532,0.205438,...,0.311707,0.427835,1.704787,0.170267,0.136812,0.402062,1.053082,0.313190,0.430025,1.543410
4,Arizona,2022,Pac-12,0.271661,0.472081,1.379341,0.673469,0.171875,3.326875,0.351341,...,0.370241,0.448578,1.726454,0.130807,0.338295,0.559036,0.998885,0.389756,0.501253,1.398465


In [19]:
for year in range(2015,2023):
    gamelist = DataFunctions.get_fbs_games(api_instance=api_instance,year=year)
    games_df = DataFunctions.df_from_games(gamelist=gamelist)
    
    betting_list=DataFunctions.get_fbs_betting(api_instance=api_instance1,year=year,conferences=conferences)
    betting_df=DataFunctions.df_betting_lines(betting_list)
    
    teamstats = api_instance2.get_advanced_team_season_stats(year=year)
    stats_df = DataFunctions.df_team_advstats(teamstats=teamstats)
    stats_df = DataFunctions.df_stats_needed(stats_df,cols)
    
    conn = sqlite3.connect("CollegeFootball.db")
    games_df.to_sql("games",conn,if_exists="append",index=False)
    betting_df.to_sql("betting_lines",conn,if_exists="append",index=False)
    stats_df.to_sql("stats",conn,if_exists="append",index=False)
    
    conn.close()
    

In [22]:
cmd=\
"""
SELECT * 
FROM games G
INNER JOIN betting_lines B ON G.id=B.id
INNER JOIN stats S on S.team=G.home_team OR S.team=G.away_team
"""

conn=sqlite3.connect("CollegeFootball.db")
df_merged=pd.read_sql_query(cmd,conn)
conn.close() # do not forget this after query

In [23]:
df_merged.head()

,id,season,neutral_site,home_id,home_team,home_conference,home_points,away_id,away_team,away_conference,...,Offensive passing_plays ppa,Offensive passing_plays success_rate,Offensive passing_plays explosiveness,Defensive havoc total,Defensive rushing_plays ppa,Defensive rushing_plays success_rate,Defensive rushing_plays explosiveness,Defensive passing_plays ppa,Defensive passing_plays success_rate,Defensive passing_plays explosiveness
0,400603840,2015,1,2579,South Carolina,SEC,17,153,North Carolina,ACC,...,0.424908,0.470449,1.739379,0.015260,0.247354,0.499230,0.956353,0.120597,0.407489,1.455767
1,400603840,2015,1,2579,South Carolina,SEC,17,153,North Carolina,ACC,...,0.209040,0.376344,1.761884,0.013937,0.263421,0.471193,1.105939,0.239993,0.450276,1.413566
2,400603840,2015,1,2579,South Carolina,SEC,17,153,North Carolina,ACC,...,0.437672,0.452431,1.729101,0.144016,0.166638,0.481600,0.853015,0.199122,0.414286,1.525897
3,400603840,2015,1,2579,South Carolina,SEC,17,153,North Carolina,ACC,...,0.137644,0.375854,1.532951,0.152632,0.163844,0.459048,0.945961,0.182084,0.438257,1.530494
4,400603840,2015,1,2579,South Carolina,SEC,17,153,North Carolina,ACC,...,0.146938,0.382488,1.611689,0.168889,0.183018,0.455268,1.013484,0.259303,0.389034,1.705994


In [24]:
df_merged.columns

Index(['id', 'season', 'neutral_site', 'home_id', 'home_team',
       'home_conference', 'home_points', 'away_id', 'away_team',
       'away_conference', 'away_points', 'game_spread', 'game_totalpts', 'id',
       'av_spread', 'av_total', 'team', 'season', 'conference',
       'Offensive ppa', 'Offensive success_rate', 'Offensive explosiveness',
       'Offensive power_success', 'Offensive stuff_rate',
       'Offensive line_yards', 'Defensive ppa', 'Defensive success_rate',
       'Defensive explosiveness', 'Defensive power_success',
       'Defensive stuff_rate', 'Defensive line_yards', 'Offensive havoc total',
       'Offensive rushing_plays ppa', 'Offensive rushing_plays success_rate',
       'Offensive rushing_plays explosiveness', 'Offensive passing_plays ppa',
       'Offensive passing_plays success_rate',
       'Offensive passing_plays explosiveness', 'Defensive havoc total',
       'Defensive rushing_plays ppa', 'Defensive rushing_plays success_rate',
       'Defensive rushin

In [ ]:
#sample merge dataframe

# cmd=\
# """
# SELECT G.away_team,G.away_points,G.home_points,G.home_team,B.av_spread,B.av_total
# FROM games G
# INNER JOIN betting_lines B ON G.id=B.id
# """
# conn=sqlite3.connect("CollegeFootball.db")
# test_merge=pd.read_sql_query(cmd,conn)
# conn.close() # do not forget this after query
# test_merge

Merging tables to set up for NN

In [ ]:
# things we are missing for neural network: Strength of Schedule
# either grab it from somewhere, create it using our own data or use the 
# data that takes it into account.

In [ ]:
betting_list=DataFunctions.get_fbs_betting(api_instance=api_instance1,year=2017,conferences=conferences)

In [ ]:
betting_list

In [ ]:
betting_lines=[game.to_dict() for game in betting_list]
    
for game in betting_lines:
    away_length=len(game['away_team'])
    home_length=len(game['home_team'])
    game_lines=[]
    over_unders=[]
    
    for i in range(len(game['lines'])):
        if (game['lines'][i]['formatted_spread'][:away_length+2]==(game['away_team']+' -') 
        and game['lines'][i]['formatted_spread'][away_length+2:]!='null'):
            game_lines.append(float(game['lines'][i]['formatted_spread'][away_length+1:]))
            
        elif (game['lines'][i]['formatted_spread'][:home_length+2]==(game['home_team']+' -')
        and game['lines'][i]['formatted_spread'][home_length+2:]!='null'):
            game_lines.append(abs(float(game['lines'][i]['formatted_spread'][home_length+1:])))
        
        if (game['lines'][i]['over_under']!=None):
            over_unders.append(game['lines'][i]['over_under'])
            
        if len(game_lines)!=0:   
            game['av_spread']=np.mean(game_lines)
        if len(over_unders)!=0:
            game['av_total']=np.mean(over_unders)
    

In [ ]:
df = pd.DataFrame(betting_lines)[['id','av_spread','av_total']]
df

In [ ]:
df.loc[df['av_spread'] == 0]